In [1]:
import pandas as pd
import json

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.ticker import PercentFormatter

import math

In [9]:
matplotlib.use("pgf")
matplotlib.rcParams.update({
  'font.size': 8,
  'text.usetex': True,
  'text.latex.preamble': r'\usepackage{amsmath}',
  'pgf.texsystem': "pdflatex",
  'pgf.preamble': r'\usepackage{amsfonts}',
  #"font.family": "serif",
  #"pgf.rcfonts": False,
})

In [2]:
fig_width=6
fig_height=1.5

In [3]:
color_i = 0
colors = {
    'fibonacci': "tab:brown",
    'murmur3': "tab:pink",
    'radixspline': "tab:red",
    'triespline': "tab:purple"
}
def get_color(fun):
    global color_i
    global colors
    
    key = fun.lower()
    if key not in colors:
        options = list(mcolors.TABLEAU_COLORS) + list(mcolors.BASE_COLORS)[0:-1]
        colors[key] = options[color_i % len(options)]
        color_i += 1
    return colors[key]

def reset_colors():
    global color_i
    global colors
    color_i = 0
    colors = {}

In [4]:
results_file = f"../results.json"
with open (results_file) as file:
    data = json.load(file)
    df = pd.json_normalize(data, "benchmarks")

In [5]:
def dataset_name(d):
    return {
        'books': "amazon", 
        'fb': "facebook", 
        'gap_10': "gapped 10%", 
        'normal': "normal", 
        'osm': "open street map", 
        'seq': "sequential", 
        'uniform': "uniform", 
        'wiki': "wikipedia"
    }[d]
    
def get_dataset_order(d):
    return {ds: i for i, ds in enumerate([
        'seq',
        'gap_10',
        'normal',
        'uniform',
        'books',
        'wiki',
        'fb',
        'osm',
    ])}[d]

def human_func_name(f):
    return {
        'fibonacci': "Fibonacci",
        'murmur3': "Murmur3",
        'radixspline': "RS ($\epsilon = 4, r = 18$)",
        'triespline': "PLEX ($\epsilon = 4$)"
    }[f]

def human_kicking_name(k):
    return {
            'balanced_kicking': "Balanced",
            'biased_kicking_0': "Unbiased",
            'biased_kicking_20': "Biased ($20\%$)",
            'biased_kicking_80': "Biased ($80\%$)"
           }[k]

def get_marker(f):
    return {
        'fibonacci': "*",
        'murmur3': "+",
        'radixspline': "v",
        'triespline': "^"
    }[f]

def map_probe_dist(d):
    return {'0': "uniform", '1': "exponential"}[d]

In [6]:
label = "label"
func_name = "fn_name"
dataset = "dataset"
dataset_size = "dataset_size"
dataset_order = "dataset_order"
overallocation = "overallocation"

failed = "failed"
failed_at = "failed_at"
kicking_strategy = "kicking_strategy"
max_kick_count = "max_kick_count"
total_kick_count = "total_kick_count"
primary_key_ratio = "primary_key_ratio"
table_capacity = "table_capacity"

hashtable_bytes = "hashtable_bytes"
hashtable_bytes_per_key = "hashtable_bytes_per_key"
total_bytes = "total_bytes"
hashtable_lookup = "cpu_time"
build_time = "build_time"
build_throughput = "build_throughput"
build_ns_per_key = "build_ns_per_key"
probing_distribution = "probing_distribution"

def extract_fun_name(x):
    
    raw = x[x.find("hashing::"):x.find(">")].removeprefix("hashing::").lower()
    if raw.startswith("triespline"):
        return "triespline"
    if raw.startswith("radixspline"):
        return "radixspline"
    if raw.startswith("murmur"):
        return "murmur3"
    if raw.startswith("fibonacci"):
        return "fibonacci"

cuckoo_df = df[df["name"].str.startswith("BM_hashtable") & df["name"].str.contains("Cuckoo") & (df[failed] < 1.0)].copy(deep=True)
cuckoo_df[func_name] = cuckoo_df["name"].apply(extract_fun_name)
cuckoo_df[dataset] = cuckoo_df["label"].apply(lambda x : x.split(":")[1])
cuckoo_df[dataset_order] = cuckoo_df[dataset].apply(get_dataset_order)

cuckoo_df[kicking_strategy] = cuckoo_df["label"].apply(lambda x: x.split(":")[0].removeprefix("cuckoo_4_"))

cuckoo_df[probing_distribution] = cuckoo_df["name"].apply(lambda x: map_probe_dist(x.split('/')[-2]))
cuckoo_df[hashtable_bytes_per_key] = cuckoo_df.apply(lambda x: x[hashtable_bytes] / x[dataset_size], axis=1)
cuckoo_df[build_throughput] = cuckoo_df.apply(lambda x: x[dataset_size] / x[build_time], axis=1)
cuckoo_df[build_ns_per_key] = cuckoo_df.apply(lambda x: 10**9 * x[build_time] / x[dataset_size], axis=1)
# cuckoo_df[(cuckoo_df[kicking_strategy].str.startswith("biased")) & (cuckoo_df[func_name] == "triespline")][[func_name, dataset, dataset_size, overallocation, kicking_strategy, probing_distribution, hashtable_bytes_per_key, failed, failed_at, hashtable_lookup, build_time, build_throughput, build_ns_per_key]]
cuckoo_df[[func_name, dataset, dataset_size, overallocation, kicking_strategy, probing_distribution, hashtable_bytes_per_key, failed, failed_at, hashtable_lookup, build_time, build_throughput, build_ns_per_key]]

,fn_name,dataset,dataset_size,overallocation,kicking_strategy,probing_distribution,hashtable_bytes_per_key,failed,failed_at,cpu_time,build_time,build_throughput,build_ns_per_key
504,triespline,seq,200000000.0,1.05,balanced_kicking,uniform,21.000053,0.0,200000000.0,209.825904,36.542652,5.473056e+06,182.713261
505,triespline,gap_10,200000000.0,1.05,balanced_kicking,uniform,21.000053,0.0,200000000.0,348.496779,41.583305,4.809622e+06,207.916526
506,triespline,uniform,199999975.0,1.05,balanced_kicking,uniform,21.000053,0.0,199999975.0,403.433552,55.826234,3.582545e+06,279.131207
507,triespline,normal,199459933.0,1.05,balanced_kicking,uniform,21.000053,0.0,199459933.0,399.200299,56.189751,3.549756e+06,281.709465
508,triespline,books,200000000.0,1.05,balanced_kicking,uniform,21.000053,0.0,200000000.0,414.302521,88.497584,2.259949e+06,442.487920
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1232,radixspline,seq,200000000.0,1.10,biased_kicking_0,uniform,22.000052,0.0,200000000.0,214.493765,20.177960,9.911805e+06,100.889798
1240,radixspline,seq,200000000.0,1.15,biased_kicking_0,uniform,23.000052,0.0,200000000.0,214.295139,20.387514,9.809926e+06,101.937570
1248,radixspline,seq,200000000.0,1.05,biased_kicking_0,exponential,21.000052,0.0,200000000.0,212.787751,20.698173,9.662688e+06,103.490866
1256,radixspline,seq,200000000.0,1.10,biased_kicking_0,exponential,22.000052,0.0,200000000.0,213.514871,20.099270,9.950610e+06,100.496349


In [ ]:
def plot_pareto(data, overalloc, probing_dist):
    rows = 8
    cols = 4
    
    fig, axs = plt.subplots(rows, cols, figsize=(fig_width*0.8, fig_height*rows*0.65), sharex=True, sharey=True)
    
    data = data[(data[overallocation] == overalloc) & (data[probing_distribution] == probing_dist)].copy(deep=True).sort_values(by=[dataset_order, dataset_size])
    for i, (ds, df) in enumerate(data.groupby([dataset], sort=False)):
        pad = 7.5
        axs[i][-1].annotate(
            dataset_name(ds), 
            xy=(0, 0.5),
            xytext=(60 + axs[i][-1].yaxis.labelpad + pad, 0),
            xycoords=axs[i][-1].yaxis.label, 
            textcoords='offset points',
            size='large', 
            ha='left', 
            va='center',
            rotation=0
        )
        
        for j, (kicking_strat, df) in enumerate(df.groupby([kicking_strategy], sort=False)):
            ax = axs[i][j]                        
            if i == 0:
                ax.set_title(human_kicking_name(kicking_strat))

            ax.grid(linestyle='--', axis='both')
            ax.set_xlim([0, 520])
            ax.set_xticks([0, 150, 300, 450])
            ax.set_ylim([0, 780])
            ax.set_yticks([0, 200, 400, 600])
                
            for k, (name, df) in enumerate(df.groupby([func_name], sort=False)):
                ax.scatter(x=df[build_ns_per_key], y=df[hashtable_lookup], label=human_func_name(name), color=get_color(name), marker=get_marker(name))
                
    h, l = axs[0][0].get_legend_handles_labels()
    legend = fig.legend(
        h, l,
        loc="lower center",
        bbox_to_anchor=(0.5, 0.04),
        ncol=4,
        borderpad=0.4, 
        labelspacing=0.2, 
        handlelength=1.0, 
        handletextpad=0.5, 
        columnspacing=0.7
    )

    fig.suptitle(f"Lookup vs. Build Time ($\lambda = {overalloc}$, {probing_dist} probing)", y=0.93, fontweight="bold")
    fig.text(0.03, 0.5, 'Lookup Time (ns per key)', va='center', rotation='vertical')
    fig.text(0.5, 0.08, 'Build Time (ns per key)', ha='center')
        
    fig.savefig(f"cuckoo_pareto_{probing_dist}_{overalloc}.pdf", bbox_inches="tight", dpi=300)
    fig.savefig(f"cuckoo_pareto_{probing_dist}_{overalloc}.pgf", bbox_inches="tight", dpi=300)

for overalloc in [1.05, 1.10, 1.15]:
    for probing_dist in ["uniform"]:
        plot_pareto(cuckoo_df, overalloc=overalloc, probing_dist=probing_dist)

In [ ]:
def plot_primary_key_ratio(data, overalloc, probing_dist):
    rows = 8
    cols = 4
    
    fig, axs = plt.subplots(rows, cols, figsize=(fig_width*0.8, fig_height*rows*0.65), sharex=True, sharey=True)
    
    data = data[(data[overallocation] == overalloc) & (data[probing_distribution] == probing_dist)].copy(deep=True).sort_values(by=[dataset_order, dataset_size])
    for i, (ds, df) in enumerate(data.groupby([dataset], sort=False)):
        pad = 7.5
        axs[i][-1].annotate(
            dataset_name(ds), 
            xy=(0, 0.5),
            xytext=(60 + axs[i][-1].yaxis.labelpad + pad, 0),
            xycoords=axs[i][-1].yaxis.label, 
            textcoords='offset points',
            size='large', 
            ha='left', 
            va='center',
            rotation=0
        )
        
        for j, (kicking_strat, df) in enumerate(df.groupby([kicking_strategy], sort=False)):
            ax = axs[i][j]                        
            if i == 0:
                ax.set_title(human_kicking_name(kicking_strat))
    
            for k, (name, df) in enumerate(df.groupby([func_name], sort=False)):
                ax.scatter(x=df[primary_key_ratio], y=df[hashtable_lookup], label=human_func_name(name), color=get_color(name), marker=get_marker(name))
        
            ax.grid(linestyle='--', axis='both')
            ax.set_xticks([1.0, 0.66, 0.33, 0])
            ax.xaxis.set_major_formatter(PercentFormatter(xmax=1.0))
            ax.invert_xaxis()
            # for tick in ax.get_xticks():
            #     tick.set_rotation(90)
            ax.set_ylim([0, 780])
            ax.set_yticks([0, 200, 400, 600])
                
    h, l = axs[0][0].get_legend_handles_labels()
    legend = fig.legend(
        h, l,
        loc="lower center",
        bbox_to_anchor=(0.5, 0.04),
        ncol=4,
        borderpad=0.4, 
        labelspacing=0.2, 
        handlelength=1.0, 
        handletextpad=0.5, 
        columnspacing=0.7
    )

    fig.suptitle(f"Lookup vs. Primary Key Ratio ($\lambda = {overalloc}$, {probing_dist} probing)", y=0.93, fontweight="bold")
    fig.text(0.03, 0.5, 'Lookup Time (ns per key)', va='center', rotation='vertical')
    fig.text(0.5, 0.08, 'Primary Key Ratio (percent)', ha='center')
    fig.subplots_adjust(wspace=0.3)
        
    fig.savefig(f"cuckoo_primary_key_ratio_{probing_dist}_{overalloc}.pdf", bbox_inches="tight", dpi=300)
    fig.savefig(f"cuckoo_primary_key_ratio_{probing_dist}_{overalloc}.pgf", bbox_inches="tight", dpi=300)

for overalloc in [1.05, 1.10, 1.15]:
    for probing_dist in ["uniform", "exponential"]:
        plot_primary_key_ratio(cuckoo_df, overalloc=overalloc, probing_dist=probing_dist)